# Benchmarking

## Import Libraries

In [ ]:
import os
os.chdir('..')

from ipywidgets import interact, interact_manual
import pandas as pd
import numpy as np
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.reporting import formatter as fmt, plots
import statsmodels.api as sm

## Import bmk data

In [ ]:
bmk_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\bmk_data.xlsx', sheet_name='data_m', index_col=0)
bmk_df = bmk_df[['HFRXM Index','HFRIMI Index',
                 'SGSLRPEW Index', 'SGSLRPWH Index', 'SGIXTFXA Index', 'NEIXCTAT Index',
                 'HFRXAR Index', 'HFRXEH Index', 'HFRXEW Index', 'HFRXED Index',
                 'HFRXCA Index', 'HFRXEMC Index', 'HFRXCOM Index',
                 'SECTGLMA Index', 'SECTNEUT Index','SECTMSTR Index',
                 'HEDGGLMA Index', 'HEDGNAV Index','HEDGNEUT Index','HEDGCONV Index',
                 'CSLAB Index', 'CSLABED Index', 'CSLABGS Index', 'CSLABMF Index',
                 'CSLABT15 Index', 'CSLABT18 Index', 'CSLABT20 Index']]
#rename columns
bmk_df.columns = ['HFRX Macro/CTA', 'HFRI Macro Total Index',
                  'Eqty Risk Prem', 'Eqty Risk Prem-Hedged','SGI Cross Asset Trend', 'SG Trend Index',
                  'HFRX Abs Ret', 'HFRX Eqty Hedge', 'HFRX Equal Wgtd', 'HFRX Event Driven',
                  'HFRX RV FI Conv Arb', 'HFRX EM', 'HFRX Macro Comdty',
                  'CS AllHedge Global Macro', 'CS AllHedge Eqty Mkt Neutral','CS AllHedge Multi_Strat',
                  'CS Global Macro', 'CS Hedge Fund', 'CS Eqty Mkt Neutral', 'CS Conv Arb',
                 'CS Liq Alt Beta', 'CS Event Driven', 'CS Global Strat', 'CS Managed Fut',
                 'CS Managed Fut-15% vol','CS Managed Fut-18% vol', 'CS Managed Fut-20% vol']

#format data to returns
bmk_df = dm.format_data(bmk_df)

## Create bmk_dict

In [ ]:
#group indicies into sub portfolios
sp_ind_dict = {'Global Macro':['HFRX Macro/CTA', 'HFRI Macro Total Index','CS Global Macro'],
            'Absolute Return':['HFRX Abs Ret', 'HFRX Equal Wgtd'],
                'Trend Following':['SGI Cross Asset Trend', 'SG Trend Index','CS Managed Fut-18% vol']}
#create bmk_dict with sub portfolios as keys
bmk_dict = {}
for key in sp_ind_dict:
    bmk_dict[key] = bmk_df[sp_ind_dict[key]].copy()

## Display bmk returns

In [ ]:
@interact
def display_bmk_returns(sub_port=bmk_dict.keys()):
    return fmt.get_returns_styler(bmk_dict[sub_port])

## Import liq bmk data

In [ ]:
liq_alts_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\mgr_data_m.xlsx', sheet_name='bny',index_col=0)
liq_alts_sp = liq_alts_df[['Total Liquid Alts',  'Global Macro',  'Trend Following','Alt Risk Premia']]
liq_alts_sp = liq_alts_sp.iloc[52:,]
liq_alts_mgr = liq_alts_df[['1907 ARP EM', '1907 ARP TF', '1907 CFM ISE Risk Premia','1907 Campbell TF', '1907 III CV',
                            '1907 III Class A', '1907 Kepos RP','1907 Penso Class A', '1907 Systematica TF',
                            'ABC Reversion','Acadian Commodity AR', 'Black Pearl RP','Blueshift', 'Bridgewater Alpha',
                            'DE Shaw Oculus Fund', 'Duality','Element Capital', 'Elliott', 'One River Trend']]

## Create liq_alts_dict

In [ ]:
sp_mgr_dict = {'Global Macro':['1907 Penso Class A', 'Bridgewater Alpha','DE Shaw Oculus Fund', 'Element Capital', 'Elliott'],
                'Absolute Return':['1907 ARP EM','1907 III Class A', '1907 Kepos RP',
                            'ABC Reversion','Blueshift', 'Duality'],
                'Trend Following':['1907 ARP TF','1907 Campbell TF', '1907 Systematica TF', 'One River Trend', 'Trend Following']}

liq_alts_dict = {}
for key in sp_mgr_dict:
    liq_alts_dict[key] = liq_alts_df[sp_mgr_dict[key]].copy()
    liq_alts_dict[key].dropna(inplace=True)

for key in liq_alts_dict:
    liq_alts_dict[key][key] = liq_alts_dict[key].sum(axis=1)/len(liq_alts_dict[key].columns)

## Display Liq Alts Returns

In [ ]:
@interact
def display_liq_alts_returns(sub_port=liq_alts_dict.keys()):
    return fmt.get_returns_styler(liq_alts_dict[sub_port])

# Run Multiple Linear Regression

In [ ]:
reg_results_dict = {}

for key in liq_alts_dict:
    temp_liq_alts_df = liq_alts_dict[key].copy()
    temp_bmk_df = bmk_dict[key].copy()
    sp_results_dict = {}
    for col in temp_liq_alts_df.columns:

        data = dm.merge_data_frames(temp_liq_alts_df[[col]],temp_bmk_df)
        x = data[list(temp_bmk_df.columns)]
        x = x.to_numpy()

        y = data[[col]].copy()
        y.dropna(inplace=True)
        y = np.array(list(y[col]))

        x, y = np.array(x), np.array(y)
        x = sm.add_constant(x)

        model = sm.OLS(y, x)
        sp_results_dict[col] = model.fit()
    reg_results_dict[key] = sp_results_dict


## Display Global Macro Results

In [ ]:
sp_gm = 'Global Macro'
@interact
def display_gm_results(manager = reg_results_dict[sp_gm].keys()):
    return print(reg_results_dict[sp_gm][manager].summary())


## Display Global Macro Predictor Keys

In [ ]:
for n in range(len(bmk_dict[sp_gm].columns)):
    print('x{}: {}'.format(n+1,bmk_dict[sp_gm].columns[n]))

# Absolute Return
## Display Absolute Return Results

In [ ]:
sp_ar = 'Absolute Return'
@interact
def display_ar_results(manager = reg_results_dict[sp_ar].keys()):
    return print(reg_results_dict[sp_ar][manager].summary())


## Display Predictor Keys

In [ ]:
for n in range(len(bmk_dict[sp_ar].columns)):
    print('x{}: {}'.format(n+1,bmk_dict[sp_ar].columns[n]))

# Trend Following
## Display Trend Following Results

In [ ]:
sp_tf = 'Trend Following'
@interact
def display_gm_results(manager = reg_results_dict[sp_tf].keys()):
    return print(reg_results_dict[sp_tf][manager].summary())


## Display Predictor Keys

In [ ]:
for n in range(len(bmk_dict[sp_tf].columns)):
    print('x{}: {}'.format(n+1,bmk_dict[sp_tf].columns[n]))